## Imports

In [2]:
import numpy as np
import pandas as pd
import pywt 
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math
from pandas.api.types import CategoricalDtype
from collections import Counter
import plotly.io as pio

In [3]:
pio.kaleido.scope.default_format = "pdf"

#### Task Evoked Pupillary Response (TEPR)

Task evoked pupillary response (TEPR) is calculated after correcting for luminance-induced pupil dilation: $𝑇𝐸𝑃𝑅 = 𝑑_m − 𝑑(𝑌)$, where $d_m$ is the measured pupil dilation, and $d(Y)$ is the predicted pupil dilation for the given luminance level. 

Predicted pupil dilation is calculated from a calibration sequence that produces and individual mapping model for each participant. The calibration sequence consists of 8 solid gray colors with varying luminance levels displayed in a psuedo-random order for 6 seconds each. The luminance levels span the range from 0.0 to 0.78, and for each calibration level, the first 0.5s of data is discarded to account for the initial pupillary response to the change in luminance, which can take a maximum of 0.5s. . The individual mapping model is calculated using a non-linear least squares regression to fit the equation $𝑑(𝑌) = 𝑎 · 𝑒^{−𝑏·𝑌} + c$ to the measured pupil dilation data for each participant. 

Pupil dilation data and the average luminance data were collected at 90 Hz, the display rate of the HMD.

See: Eckert, M., Robotham, T., Habets, E. A. P., and Rummukainen, O. S. (2022). Pupillary Light Reflex Correction for Robust Pupillometry in Virtual Reality. Proc. ACM Comput. Graph. Interact. Tech. 5, 1–16. doi: 10.1145/3530798

In [4]:
def pupil_func(x, a, b, c):
    return a * np.exp(-b * x) + c

#### Index of Pupillary Activity (IPA)

The frequency of pupil diameter oscilation over time is an indicator of cognitive load measured as the index of pupillary activity (IPA). It is an open-source alternative to the Index of Cognitive Activity. The IPA is implemented as described in the original paper, using a wavelet decomposition of the pupil diameter. The wavelet function is symlet4, because the signal was sampled at 90 Hz (rather than 250 Hz, as in the original paper).  

See: 
Duchowski, A. T., Krejtz, K., Krejtz, I., Biele, C., Niedzielska, A., Kiefer, P., Raubal, M., and Giannopoulos, I. (2018). The Index of Pupillary Activity: Measuring Cognitive Load vis-à-vis Task Difficulty with Pupil Oscillation. Proc. ACM Hum.-Comput. Interact. 2, 282:1–282:13. doi: 10.1145/3173574.3173856

In [5]:
def modmax(d):
    # compute signal modulus
    m = [0.0]*len(d)
    for i in range(len(d)):
        m[i] = math.fabs(d[i])
    # if value is larger than both neighbours, and strictly larger than either, then it is a local maximum
    t = [0.0]*len(d)
    for i in range(len(d)):
        ll = m[i -1] if i >= 1 else m[i]
        oo = m[i]
        rr = m[i+1] if i < len(d)-2 else m[i]
        if (ll <= oo and oo >= rr) and (ll < oo or oo > rr):
        # compute magnitude
            t[i] = math.sqrt(d[i]**2)
        else:
            t[i] = 0.0
    return t

In [6]:
def ipa_func(d):
    # obtain 2-level DWT of pupil diameter signal d
    try:
        (cA2 ,cD2 ,cD1) = pywt.wavedec(d,'sym4', 'per', level=2)
    except ValueError :
        return
    # get signal duration (in seconds)
    tt = d.index[-1] - d.index[0]

    # normalize by 1/2 j , j = 2 for 2-level DWT
    cA2 [:] = [x / math.sqrt (4.0) for x in cA2]
    cD1 [:] = [x / math.sqrt (2.0) for x in cD1]
    cD2 [:] = [x / math.sqrt (4.0) for x in cD2]

    # detect modulus maxima
    cD2m = modmax(cD2)

    # threshold using universal threshold λuniv = σˆp(2logn)
    # where σˆ is the standard deviation of the noise
    univ = np.std(cD2m) * math.sqrt (2.0* np.log2(len(cD2m )))
    cD2t = pywt.threshold(cD2m, univ, mode="hard")

    # compute IPA
    ctr = 0
    for i in range(len(cD2t )):
        if math.fabs(cD2t[i]) > 0: ctr += 1
    IPA = float(ctr)/tt.total_seconds()

    return IPA

## Statistical Helper Functions

In [7]:
def iqr_outlier_indices(data):
    q1 = data.quantile(.25)
    q3 = data.quantile(.75)
    iqr = stats.iqr(data, nan_policy='omit', rng=(25, 75))
    return np.where((data < (q1 - 1.5 * iqr)) | (data > (q3 + 1.5 * iqr)))

In [8]:
def iqr_stats(data):
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = stats.iqr(data, nan_policy='omit', rng=(25, 75))
    return iqr, q1, q3

In [9]:
def get_results_colors(np, wp, tp):
    pastels = px.colors.qualitative.Pastel2
    default_color = 'white'
    significant_color = pastels[0]
    non_significant_color = pastels[3]

    normal_color = non_significant_color if np < 0.05 else significant_color
    wilcox_color = default_color
    ttest_color = default_color
    if np < 0.05:
        wilcox_color = significant_color if wp < 0.05 else non_significant_color
    else:
        ttest_color = significant_color if tp < 0.05 else non_significant_color

    fill_color = [[default_color, default_color, default_color],
                  [default_color, wilcox_color, ttest_color] , 
                  [normal_color, wilcox_color, ttest_color]]
    
    return fill_color

In [10]:
def calculate_entropy(signal):
    coeff = pywt.wavedec(signal, 'sym8', 'per')
    counter_values = Counter(coeff).most_common()
    probabilities = [elem[1]/len(coeff) for elem in counter_values]
    entropy= stats.entropy(probabilities)
    return entropy

In [11]:
color_uni = '#785EF0'
color_bi = '#FFB000'
color_4dof = '#648FFF'
color_6dof = '#FE6100'


## Import Data

In [12]:
user_data_nav = pd.read_pickle('user_data_nav.pkl')
user_data_crt = pd.read_pickle('user_data_crt.pkl')
participant_data = pd.read_pickle('participant_data.pkl')

### Dataframe Structure

In [13]:
user_data_nav.head()

trial_id task_trial_id  luminance  pupilDiameter  \
id  block model method                                                    
108 0     A     4DoF        3.0           3.0   0.224356       4.866458   
                6DoF        1.0           5.0   0.225511       4.359483   
          B     4DoF        2.0           2.0   0.242239       4.650883   
                6DoF        3.0           7.0   0.256185       4.437137   
          C     4DoF        0.0           0.0   0.223505       4.658904   

                        pupil_lum_base  adj_pupil  total_time target_source  \
id  block model method                                                        
108 0     A     4DoF          3.831577   1.034881     37.0822             1   
                6DoF          3.829147   0.530336     61.0043             2   
          B     4DoF          3.771303   0.879580     67.7005             1   
                6DoF          3.746679   0.690458     96.4316             2   
          C     4DoF          3.852900   0.806004     88.0356             1   

                       creation_method       IPA  discomfort  seq  
id  block model method                                             
108 0     A     4DoF         unimanual  0.242846           2    0  
                6DoF          bimanual  0.114788           1    0  
          B     4DoF         unimanual  0.162532           1    0  
                6DoF          bimanual  0.124469           1    0  
          C     4DoF         unimanual  0.136342           1    1

In [14]:
user_data_nav.describe()

,luminance,pupilDiameter,pupil_lum_base,adj_pupil,total_time,IPA,discomfort,seq
count,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000
mean,0.227217,3.659042,3.214781,0.444261,67.618892,0.175448,2.461538,0.657343
std,0.021321,0.526402,0.474457,0.301660,28.407818,0.056267,2.527794,0.851046
min,0.158152,2.448724,2.170120,-0.354323,24.187100,0.044233,0.000000,0.000000
25%,0.215047,3.276390,2.955237,0.214713,48.545975,0.136329,0.000000,0.000000
50%,0.225265,3.615765,3.173565,0.428035,58.260850,0.170063,2.000000,0.000000
75%,0.239252,4.040277,3.445601,0.675510,77.249224,0.212548,4.000000,1.000000
max,0.294793,5.205173,4.245193,1.268353,212.109400,0.308142,10.000000,4.000000


In [15]:
user_data_crt.head()

trial_id task_trial_id  luminance  pupilDiameter  \
id  block model method                                                       
108 0     A     bimanual       2.0           6.0   0.280843       4.402825   
                unimanual      0.0           0.0   0.259386       5.104381   
          B     bimanual       0.0           4.0   0.320817       4.189030   
                unimanual      1.0           1.0   0.263313       4.835740   
          C     bimanual       1.0           5.0   0.237598       4.375924   

                           pupil_lum_base  adj_pupil  total_time target_id  \
id  block model method                                                       
108 0     A     bimanual         3.651643   0.751182     98.3030         2   
                unimanual        3.717137   1.387243     62.7652         1   
          B     bimanual         3.540984   0.648046     95.2104         2   
                unimanual        3.714536   1.121204     40.4416         1   
          C     bimanual         3.783636   0.592288     36.4409         2   

                                IPA  seq  PointPlaced  Move  Draw  Erase  \
id  block model method                                                     
108 0     A     bimanual   0.111912    1           19     6     0      0   
                unimanual  0.107009    2            0     0     4      0   
          B     bimanual   0.115569    2           17     3     0      0   
                unimanual  0.098936    2            0     0     4      0   
          C     bimanual   0.164693    1           12     1     0      0   

                           PointDeleted  
id  block model method                   
108 0     A     bimanual              0  
                unimanual             0  
          B     bimanual              0  
                unimanual             0  
          C     bimanual              0

In [16]:
user_data_crt.describe()

,luminance,pupilDiameter,pupil_lum_base,adj_pupil,total_time,IPA,seq,PointPlaced,Move,Draw,Erase,PointDeleted
count,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000
mean,0.245947,3.732538,3.183800,0.548739,58.350967,0.175554,0.777778,5.000000,3.559028,2.541667,0.611111,0.243056
std,0.036211,0.592989,0.478283,0.328533,41.573572,0.064036,0.976693,6.780789,7.605013,6.143323,2.307724,0.853331
min,0.164554,2.419907,2.162557,-0.141567,11.261400,0.048284,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.215910,3.259171,2.915578,0.300946,31.700625,0.125672,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.244383,3.740561,3.111785,0.532538,46.113950,0.166854,0.000000,2.500000,0.000000,0.500000,0.000000,0.000000
75%,0.272687,4.189731,3.437178,0.748840,70.512075,0.214618,1.000000,8.000000,2.000000,2.000000,0.000000,0.000000
max,0.336842,5.373014,4.248593,1.713309,371.058100,0.401104,4.000000,50.000000,38.000000,61.000000,19.000000,6.000000


In [17]:
participant_data.head()

,id,block,age,sex,hand,motion_sick,pre_ssq,post_ssq,delta_ssq,crt_pref,nav_pref
0,108,0,18 - 24,Male,Right-handed,No,33.66,26.18,-7.48,Path drawing,Pointing (6 DoF)
1,109,1,18 - 24,Male,Right-handed,No,0.00,0.00,0.00,Path drawing,Pointing (6 DoF)
2,110,2,18 - 24,Female,Right-handed,Yes,0.00,52.36,52.36,Point placing with plane,Pointing (6 DoF)
3,111,3,18 - 24,Male,Left-handed,No,3.74,14.96,11.22,Path drawing,Forward/Backward (4 DoF)
4,112,0,18 - 24,Male,Right-handed,Yes,3.74,29.92,26.18,Point placing with plane,Pointing (6 DoF)


In [18]:
participant_data.describe()

,id,block,pre_ssq,post_ssq,delta_ssq
count,36.000000,36.000000,36.000000,36.000000,36.00000
mean,129.388889,1.500000,7.999444,25.764444,17.76500
std,13.032438,1.133893,11.278717,26.214748,26.58709
min,108.000000,0.000000,0.000000,0.000000,-41.14000
25%,118.750000,0.750000,0.000000,6.545000,0.00000
50%,129.500000,1.500000,3.740000,14.960000,9.35000
75%,140.250000,2.250000,11.220000,33.660000,29.92000
max,151.000000,3.000000,48.620000,97.240000,82.28000


## Statistical Analysis

#### Participant Demographics

In [19]:
demographics_sex = px.pie(participant_data, names = participant_data['sex'], title='Sex', width=500, height=400)
demographics_sex.update_traces(textposition='inside', textinfo='percent+label')
demographics_sex.show()

demographics_age = px.pie(participant_data, names = participant_data['age'], title='Age', width=500, height=400)
demographics_age.update_traces(textinfo='percent+label')
demographics_age.show()

### Navigation Workload

#### IPA Evaluation

In [20]:
ipa_nav = user_data_nav.loc[(slice(None), slice(None), slice(None)), 'IPA']
ipa_nav = ipa_nav.unstack(level=(3))

outliers_4dof = iqr_outlier_indices(ipa_nav['4DoF'])
ipa_nav = ipa_nav.drop(ipa_nav.iloc[outliers_4dof[0]].index)

outliers_6dof = iqr_outlier_indices(ipa_nav['6DoF'])
ipa_nav = ipa_nav.drop(ipa_nav.iloc[outliers_6dof[0]].index)

ipa_4dof = ipa_nav['4DoF']
ipa_6dof = ipa_nav['6DoF']

n_stat, n_p = stats.shapiro(ipa_nav['6DoF'] - ipa_nav['4DoF'])
w_stat, w_p = stats.wilcoxon(ipa_4dof, ipa_6dof, nan_policy='omit')
t_stat, t_p = stats.ttest_rel(ipa_4dof, ipa_6dof, nan_policy='omit')

results_colors = get_results_colors(n_p, w_p, t_p)
results_header = ['<b>Test</b>', '<b>Result</b>', '<b>p-value</b>', '<b>effect size</b>']
results_values=[['Shapiro-Wilk', 'Wilcoxon', 'T-test'], [n_stat, w_stat, t_stat], [n_p, w_p, t_p]]
results_prefix = ['','', 'p = ', '']
fill_color = results_colors

if n_p < 0.05:
    iqr_4dof, q1_4dof, q3_4dof = iqr_stats(ipa_4dof)
    iqr_6dof, q1_6dof, q3_6dof = iqr_stats(ipa_6dof)
    summary_header = ['', '<b>Median</b>', '<b>IQR</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>4-DoF</b>', '<b>6-DoF</b>'], [ipa_4dof.median(), ipa_6dof.median()], [iqr_4dof, iqr_6dof], [ipa_4dof.min(), ipa_6dof.min()], [ipa_4dof.max(), ipa_6dof.max()], [ipa_4dof.skew(), ipa_6dof.skew()]]
    

else:
    summary_header = ['', '<b>Mean</b>', '<b>STD</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>4-DoF</b>', '<b>6-DoF</b>'], [ipa_4dof.mean(), ipa_6dof.mean()], [ipa_4dof.std(), ipa_6dof.std()], [ipa_4dof.min(), ipa_6dof.min()], [ipa_4dof.max(), ipa_6dof.max()], [ipa_4dof.skew(), ipa_6dof.skew()]]

In [21]:
stats_table = go.Table(columnwidth=[5,4,4,4,4])
stats_table.header = dict(
    values=summary_header, 
    align = ['right', 'center'],)
stats_table.cells = dict(
        values=summary_values,
        align = ['right', 'center'],
        format=["", ".3f"],
        )

results_table = go.Table(columnwidth=[5,4,4, 4])
results_table.header = dict(
    values= results_header, 
    align = ['right', 'center'],)
results_table.cells = dict(
        values=results_values,
        align = ['right', 'center'],
        format=["", ".3f", ".5f", ".3f"],
        fill_color=fill_color,
        prefix=results_prefix,
        )

iqr_fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=False,
    specs=[
        [{"type": "table"}, {"type": "box", "rowspan": 2}],
        [{"type": "table"}, None]
    ])

iqr_fig.add_trace(stats_table, row=2, col=1)
iqr_fig.add_trace(go.Box(y=ipa_nav['4DoF'], name="4-DoF", notched=True, notchwidth=0.15), row=1, col=2)
iqr_fig.add_trace(go.Box(y=ipa_nav['6DoF'], name="6-DoF", notched=True, notchwidth=0.15), row=1, col=2)
iqr_fig.add_trace(results_table, row=1, col=1)

iqr_fig.update_layout(
    title_text="IPA by Navigation Method",
    width=1200,
    xaxis_title='Method',
    yaxis_title='IPA',
)

iqr_fig.update_yaxes(range=[0,max(1.1*ipa_4dof.max(), 1.1*ipa_6dof.max())])
iqr_fig.show()

In [22]:
ipa_nav_fig = go.Figure(
    layout=go.Layout(
        title = dict(
            text="<b>IPA by Navigation Method</b>",
            xanchor='center',
            yanchor='top',
            x=0.5,
            y=0.945,
            font=dict(size=22),
            ),
        xaxis=dict(title="<b>Traversal Method</b>"),
        yaxis=dict(title="<b>IPA</b>"),
        height=540,
        width=720,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
            ),
        font=dict(
            family="Open Sans",
            size=14,
        ),
    )
)
ipa_nav_fig.add_trace(go.Box(y=ipa_nav['4DoF'], name="<i>4-DoF</i>", notched=True, notchwidth=0.25, marker_color=color_4dof,))
ipa_nav_fig.add_trace(go.Box(y=ipa_nav['6DoF'], name="<i>6-DoF</i>", notched=True, notchwidth=0.25, marker_color=color_6dof, ))
ipa_nav_fig.update_traces(boxmean='sd')

ipa_nav_fig.update_yaxes(range=[0, 0.35])

In [23]:
ipa_nav_fig.write_image("Plots/4_ipa_nav.pdf", engine="kaleido" )

### Creation Workload

#### IPA Evaluation

In [24]:
ipa_crt = user_data_crt.loc[(slice(None), slice(None), slice(None)), 'IPA']
ipa_crt = ipa_crt.unstack(level=(3))

uni_outliers = iqr_outlier_indices(ipa_crt['unimanual'])
ipa_crt = ipa_crt.drop(ipa_crt.iloc[uni_outliers].index)

bi_outliers = iqr_outlier_indices(ipa_crt['bimanual'])
ipa_crt = ipa_crt.drop(ipa_crt.iloc[bi_outliers].index)

ipa_uni = ipa_crt['unimanual']
ipa_bi = ipa_crt['bimanual']

n_stat, n_p = stats.shapiro(ipa_uni - ipa_bi)
w_stat, w_p = stats.wilcoxon(ipa_uni, ipa_bi)
t_stat, t_p = stats.ttest_rel(ipa_uni, ipa_bi)

results_colors = get_results_colors(n_p, w_p, t_p)
results_header = ['<b>Test</b>', '<b>Result</b>', '<b>p-value</b>']
results_values=[['Shapiro-Wilk', 'Wilcoxon', 'T-test'], [n_stat, w_stat, t_stat], [n_p, w_p, t_p]]
results_prefix = ['','', 'p = ',]
fill_color = results_colors

if n_p < 0.05:
    uni_iqr, uni_q1, uni_q3 = iqr_stats(ipa_uni)
    bi_iqr, bi_q1, bi_q3 = iqr_stats(ipa_bi)
    summary_header = ['', '<b>Median</b>', '<b>IQR</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>Path Drawing</b>', '<b>Point Placing</b>'], [ipa_uni.median(), ipa_bi.median()], [uni_iqr, bi_iqr], [ipa_uni.min(), ipa_bi.min()], [ipa_uni.max(), ipa_bi.max()], [ipa_uni.skew(), ipa_bi.skew()]]

else:
    summary_header = ['', '<b>Mean</b>', '<b>STD</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>Path Drawing</b>', '<b>Point Placing</b>'], [ipa_uni.mean(), ipa_bi.mean()], [ipa_uni.std(), ipa_bi.std()], [ipa_uni.min(), ipa_bi.min()], [ipa_uni.max(), ipa_bi.max()], [ipa_uni.skew(), ipa_bi.skew()]]


In [25]:
stats_table = go.Table(columnwidth=[5,4,4,4,4], name='Summary')
stats_table.header = dict(
    values=summary_header, 
    align = ['right', 'center'],)
stats_table.cells = dict(
        values=summary_values,
        align = ['right', 'center'],
        format=["", ".3f"],
        )

results_table = go.Table(columnwidth=[5,4,4])
results_table.header = dict(
    values= results_header, 
    align = ['right', 'center'],)
results_table.cells = dict(
        values=results_values,
        align = ['right', 'center'],
        format=["", ".2f", ".5f"],
        fill_color=fill_color,
        prefix=results_prefix,
        )

ipa_fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=False,
    specs=[
        [{"type": "table"}, {"type": "box", "rowspan": 2}],
        [{"type": "table"}, None]
    ])

ipa_fig.add_trace(stats_table, row=2, col=1)
ipa_fig.add_trace(go.Box(y=ipa_crt['bimanual'], name="Point Placing", notched=True, notchwidth=0.15), row=1, col=2)
ipa_fig.add_trace(go.Box(y=ipa_crt['unimanual'], name="Path Drawing", notched=True, notchwidth=0.15), row=1, col=2)
ipa_fig.add_trace(results_table, row=1, col=1)

ipa_fig.update_layout(
    title_text="IPA by Creation Method",
    width=1200,
    xaxis_title='Method',
    yaxis_title='IPA',
)

ipa_max = max(ipa_uni.max(), ipa_bi.max())
ipa_min = min(ipa_uni.min(), ipa_bi.min())
ipa_min = 0 if ipa_min > 0 else ipa_min
ipa_range = 0.025 * (ipa_max - ipa_min)
ipa_fig.update_yaxes(range=[0,max(1.1*ipa_uni.max(), 1.1*ipa_bi.max())])
ipa_fig.show()

In [26]:
ipa_crt_fig = go.Figure(
    layout=go.Layout(
        title = dict(
            text="<b>IPA by Creation Method</b>",
            xanchor='center',
            yanchor='top',
            x=0.5,
            y=0.945,
            font=dict(size=22),
            ),
        xaxis=dict(title="<b>Creation Method</b>"),
        yaxis=dict(title="<b>IPA</b>"),
        height=540,
        width=720,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
            ),
        font=dict(
            family="Open Sans",
            size=14,
        ),
    )
)
ipa_crt_fig.add_trace(go.Box(y=ipa_crt['unimanual'], name='Path Drawing', notched=True, notchwidth=0.25, marker_color=color_uni, ))
ipa_crt_fig.add_trace(go.Box(y=ipa_crt['bimanual'], name='Point Placing', notched=True, notchwidth=0.25, marker_color=color_bi, ))
ipa_crt_fig.update_traces(boxmean='sd')
ipa_crt_fig.update_yaxes(range=[0, 0.35])

In [27]:
ipa_crt_fig.write_image("Plots/4_ipa_crt.pdf", engine="kaleido" )

### Survey Analysis

In [28]:
discomfort = user_data_nav.loc[(slice(None), slice(None), slice(None)), 'discomfort']
discomfort = discomfort.unstack(level=(3))

first_4dof = discomfort['4DoF'].groupby(level=0).first()
first_6dof = discomfort['6DoF'].groupby(level=0).first()

last_4dof = discomfort['4DoF'].groupby(level=0).last()
last_6dof = discomfort['6DoF'].groupby(level=0).last()

mean_4dof = discomfort['4DoF'].groupby(level=0).mean()
mean_6dof = discomfort['6DoF'].groupby(level=0).mean()

discomfort_4dof = last_4dof #- first_4dof
discomfort_6dof = last_6dof #- first_6dof

delta_discomfort = discomfort_6dof - discomfort_4dof

n_stat, n_p = stats.shapiro(discomfort.stack())
w_stat, w_p = stats.wilcoxon(delta_discomfort, nan_policy='raise', zero_method='wilcox')
t_stat, t_p = stats.ttest_rel(discomfort_4dof, discomfort_6dof, nan_policy='raise')

results_colors = get_results_colors(n_p, w_p, t_p)
results_header = ['<b>Test</b>', '<b>Result</b>', '<b>p-value</b>']
results_values=[['Shapiro-Wilk', 'Wilcoxon', 'T-test'], [n_stat, w_stat, t_stat], [n_p, w_p, t_p]]
results_prefix = ['','', 'p = ',]
fill_color = results_colors 

if n_p < 0.05:
    iqr_4dof, q1_4dof, q3_4dof = iqr_stats(discomfort_4dof)
    iqr_6dof, q1_6dof, q3_6dof = iqr_stats(discomfort_6dof)
    summary_header = ['', '<b>Median</b>', '<b>IQR</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>4-DoF</b>', '<b>6-DoF</b>'], [discomfort_4dof.median(), discomfort_6dof.median()], [iqr_4dof, iqr_6dof], [discomfort_4dof.min(), discomfort_6dof.min()], [discomfort_4dof.max(), discomfort_6dof.max()], [discomfort_4dof.skew(), discomfort_6dof.skew()]]

else:
    summary_header = ['', '<b>Mean</b>', '<b>STD</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>4-DoF</b>', '<b>6-DoF</b>'], [discomfort_4dof.mean(), discomfort_6dof.mean()], [discomfort_4dof.std(), discomfort_6dof.std()], [discomfort_4dof.min(), discomfort_6dof.min()], [discomfort_4dof.max(), discomfort_6dof.max()], [discomfort_4dof.skew(), discomfort_6dof.skew()]]

In [29]:
stats_table = go.Table(columnwidth=[5,4,4,4,4])
stats_table.header = dict(
    values=summary_header, 
    align = ['right', 'center'],)
stats_table.cells = dict(
        values=summary_values,
        align = ['right', 'center'],
        format=["", ".3f"],
        )

results_table = go.Table(columnwidth=[5,4,4])
results_table.header = dict(
    values= results_header, 
    align = ['right', 'center'],)
results_table.cells = dict(
        values=results_values,
        align = ['right', 'center'],
        format=["", ".2f", ".5f"],
        fill_color=fill_color,
        prefix=results_prefix,
        )

discomfort_fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=False,
    specs=[
        [{"type": "table"}, {"type": "box", "rowspan": 2}],
        [{"type": "table"}, None]
    ])

discomfort_fig.add_trace(stats_table, row=2, col=1)
discomfort_fig.add_trace(go.Box(y=discomfort_4dof, name="4-DoF", notched=False, notchwidth=0.05), row=1, col=2)
discomfort_fig.add_trace(go.Box(y=discomfort_6dof, name="6-DoF", notched=False, notchwidth=0.05), row=1, col=2)
discomfort_fig.add_trace(results_table, row=1, col=1)

discomfort_fig.update_layout(
    title_text="Discomfort by Navigation Method",
    width=1200,
    xaxis_title='Method',
    yaxis_title='Discomfort',
)

discomfort_fig.update_yaxes(range=[0, 10])
discomfort_fig.show()

In [30]:
seq_ds_fig = go.Figure(
        layout=go.Layout(
        title = dict(
            text="<b>Discomfort Score by Navigation Method</b>",
            xanchor='center',
            yanchor='top',
            x=0.5,
            y=0.945,
            font=dict(size=22),
            ),
        xaxis=dict(title="<b>Navigation Method</b>"),
        yaxis=dict(title="<b>Discomfort Score</b>"),
        height=540,
        width=720,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
            ),
        font=dict(
            family="Open Sans",
            size=14,
        ),
    )
)
seq_ds_fig.add_trace(go.Box(y=discomfort_4dof, name="4-DoF", notched=True, notchwidth=0.25, marker_color=color_4dof,))
seq_ds_fig.add_trace(go.Box(y=discomfort_6dof, name="6-DoF", notched=True, notchwidth=0.25, marker_color=color_6dof, ))
seq_ds_fig.update_yaxes(range=[0,10])

In [31]:
seq_ds_fig.write_image("Plots/ds_nav.pdf", engine="kaleido" )

In [32]:
seq_nav = user_data_nav.loc[(slice(None), slice(None), slice(None)), 'seq']
seq_nav = seq_nav.unstack(level=(3))

seq_4dof = seq_nav['4DoF']#.groupby(level=0).mean()
seq_6dof = seq_nav['6DoF']#.groupby(level=0).mean()

seq_diff = seq_6dof - seq_4dof

n_stat, n_p = stats.shapiro(seq_nav.stack())
w_stat, w_p = stats.wilcoxon(seq_diff, nan_policy='omit', zero_method='wilcox')
t_stat, t_p = stats.ttest_rel(seq_4dof, seq_6dof, nan_policy='omit')

results_colors = get_results_colors(n_p, w_p, t_p)
results_header = ['<b>Test</b>', '<b>Result</b>', '<b>p-value</b>']
results_values=[['Shapiro-Wilk', 'Wilcoxon', 'T-test'], [n_stat, w_stat, t_stat], [n_p, w_p, t_p]]
results_prefix = ['','', 'p = ',]
fill_color = results_colors

if n_p < 0.05:
    iqr_4dof, q1_4dof, q3_4dof = iqr_stats(seq_4dof)
    iqr_6dof, q1_6dof, q3_6dof = iqr_stats(seq_6dof)
    summary_header = ['', '<b>Median</b>', '<b>IQR</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>4-DoF</b>', '<b>6-DoF</b>'], [seq_4dof.median(), seq_6dof.median()], [iqr_4dof, iqr_6dof], [seq_4dof.min(), seq_6dof.min()], [seq_4dof.max(), seq_6dof.max()], [seq_4dof.skew(), seq_6dof.skew()]]

else:
    summary_header = ['', '<b>Mean</b>', '<b>STD</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>4-DoF</b>', '<b>6-DoF</b>'], [seq_4dof.mean(), seq_6dof.mean()], [seq_4dof.std(), seq_6dof.std()], [seq_4dof.min(), seq_6dof.min()], [seq_4dof.max(), seq_6dof.max()], [seq_4dof.skew(), seq_6dof.skew()]]

In [33]:
stats_table = go.Table(columnwidth=[5,4,4,4,4])
stats_table.header = dict(
    values=summary_header, 
    align = ['right', 'center'],)
stats_table.cells = dict(
        values=summary_values,
        align = ['right', 'center'],
        format=["", ".3f"],
        )

results_table = go.Table(columnwidth=[5,4,4])
results_table.header = dict(
    values= results_header, 
    align = ['right', 'center'],)
results_table.cells = dict(
        values=results_values,
        align = ['right', 'center'],
        format=["", ".2f", ".5f"],
        fill_color=fill_color,
        prefix=results_prefix,
        )

seq_fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=False,
    specs=[
        [{"type": "table"}, {"type": "box", "rowspan": 2}],
        [{"type": "table"}, None]
    ])

seq_fig.add_trace(stats_table, row=2, col=1)
seq_fig.add_trace(go.Box(y=seq_4dof, name="4-DoF", notched=True, notchwidth=0.15), row=1, col=2)
seq_fig.add_trace(go.Box(y=seq_6dof, name="6-DoF", notched=True, notchwidth=0.15), row=1, col=2)
seq_fig.add_trace(results_table, row=1, col=1)

seq_fig.update_layout(
    title_text="SEQ by Navigation Method",
    width=1200,
    xaxis_title='Method',
    yaxis_title='SEQ',
)

seq_fig.update_yaxes(range=[0,7])
seq_fig.show()


In [34]:
seq_nav_fig = go.Figure(
        layout=go.Layout(
        title = dict(
            text="<b>SEQ by Navigation Method</b>",
            xanchor='center',
            yanchor='top',
            x=0.5,
            y=0.945,
            font=dict(size=22),
            ),
        xaxis=dict(title="<b>Navigation Method</b>"),
        yaxis=dict(title="<b>SEQ</b>"),
        height=540,
        width=720,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
            ),
        font=dict(
            family="Open Sans",
            size=14,
        ),
    )
)
seq_nav_fig.add_trace(go.Box(y=seq_4dof, name="4-DoF", notched=True, notchwidth=0.25, marker_color=color_4dof,))
seq_nav_fig.add_trace(go.Box(y=seq_6dof, name="6-DoF", notched=True, notchwidth=0.25, marker_color=color_6dof, ))
seq_nav_fig.update_yaxes(range=[0,7])

In [35]:
seq_nav_fig.write_image("Plots/seq_nav.pdf", engine="kaleido" )

In [36]:
seq_crt = user_data_crt.loc[(slice(None), slice(None), slice(None)), 'seq']
seq_crt = seq_crt.unstack(level=(3))

seq_uni = seq_crt['unimanual']
seq_bi = seq_crt['bimanual']
seq_diff = seq_uni - seq_bi

n_stat, n_p = stats.shapiro(seq_diff)
w_stat, w_p = stats.wilcoxon(seq_uni, seq_bi)
t_stat, t_p = stats.ttest_rel(seq_uni, seq_bi)

results_colors = get_results_colors(n_p, w_p, t_p)
results_header = ['<b>Test</b>', '<b>Result</b>', '<b>p-value</b>']
results_values=[['Shapiro-Wilk', 'Wilcoxon', 'T-test'], [n_stat, w_stat, t_stat], [n_p, w_p, t_p]]
results_prefix = ['','', 'p = ',]
fill_color = results_colors

if n_p < 0.05:
    iqr_uni, q1_uni, q3_uni = iqr_stats(seq_uni)
    iqr_bi, q1_bi, q3_bi = iqr_stats(seq_bi)
    summary_header = ['', '<b>Median</b>', '<b>IQR</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>Path Drawing</b>', '<b>Point Placing</b>'], [seq_uni.median(), seq_bi.median()], [iqr_uni, iqr_bi], [seq_uni.min(), seq_bi.min()], [seq_uni.max(), seq_bi.max()], [seq_uni.skew(), seq_bi.skew()]]

else:
    summary_header = ['', '<b>Mean</b>', '<b>STD</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>Path Drawing</b>', '<b>Point Placing</b>'], [seq_uni.mean(), seq_bi.mean()], [seq_uni.std(), seq_bi.std()], [seq_uni.min(), seq_bi.min()], [seq_uni.max(), seq_bi.max()], [seq_uni.skew(), seq_bi.skew()]]

In [37]:
stats_table = go.Table(columnwidth=[5,4,4,4,4])
stats_table.header = dict(
    values=summary_header, 
    align = ['right', 'center'],)
stats_table.cells = dict(
        values=summary_values,
        align = ['right', 'center'],
        format=["", ".3f"],
        )

results_table = go.Table(columnwidth=[5,4,4])
results_table.header = dict(
    values= results_header, 
    align = ['right', 'center'],)
results_table.cells = dict(
        values=results_values,
        align = ['right', 'center'],
        format=["", ".2f", ".5f"],
        fill_color=fill_color,
        prefix=results_prefix,
        )

seq_fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=False,
    specs=[
        [{"type": "table"}, {"type": "box", "rowspan": 2}],
        [{"type": "table"}, None]
    ])

seq_fig.add_trace(stats_table, row=2, col=1)
seq_fig.add_trace(go.Box(y=seq_uni, name="Path Drawing", notched=True, notchwidth=0.15), row=1, col=2)
seq_fig.add_trace(go.Box(y=seq_bi, name="Point Placing", notched=True, notchwidth=0.15), row=1, col=2)
seq_fig.add_trace(results_table, row=1, col=1)

seq_fig.update_layout(
    title_text="SEQ by Creation Method",
    width=1200,
    xaxis_title='Method',
    yaxis_title='SEQ',
)

seq_fig.update_yaxes(range=[0,7])
seq_fig.show()



In [38]:
seq_crt_fig = go.Figure(
        layout=go.Layout(
        title = dict(
            text="<b>SEQ by Creation Method</b>",
            xanchor='center',
            yanchor='top',
            x=0.5,
            y=0.945,
            font=dict(size=22),
            ),
        xaxis=dict(title="<b>Creation Method</b>"),
        yaxis=dict(title="<b>SEQ</b>"),
        height=540,
        width=720,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
            ),
        font=dict(
            family="Open Sans",
            size=14,
        ),
    )
)
seq_crt_fig.add_trace(go.Box(y=seq_uni, name='Path Drawing', notched=True, notchwidth=0.25, marker_color=color_uni, ))
seq_crt_fig.add_trace(go.Box(y=seq_bi, name="Point Placing", notched=True, notchwidth=0.25, marker_color=color_bi, ))
seq_crt_fig.update_yaxes(range=[0,7])

In [39]:
seq_crt_fig.write_image("Plots/seq_crt.pdf", engine="kaleido" )

In [40]:
crt_pref = participant_data['crt_pref'].value_counts()
nav_pref = participant_data['nav_pref'].value_counts()

pref_fig = make_subplots(
    rows=1, cols=2,
    shared_xaxes=False,
    specs=[[{'type':'domain'}, {'type':'domain'}]],
    subplot_titles=['Creation Method', 'Navigation Method'])

pref_fig.add_trace(go.Pie(
    labels = ['Path Drawing', 'Point Placing'], 
    values = crt_pref,
    textinfo='label+percent',
    name='Creation',
    legendgroup="creation", 
    legendgrouptitle_text="Creation",
    ), row=1, col=1)
pref_fig.add_trace(go.Pie(
    labels = ['4Dof', '6Dof'], 
    values = nav_pref,
    textinfo='label+percent',
    name="Navigation", 
    legendgroup="navigation", 
    legendgrouptitle_text="Navigation",), row=1, col=2)

pref_fig.update_layout(title_text='Participant Preference')
pref_fig.show()

In [41]:
crt_pref_fig = go.Figure(
        layout=go.Layout(
        title = dict(
            text="<b>Participant Creation Method Preference</b>",
            xanchor='center',
            yanchor='top',
            x=0.5,
            y=0.945,
            font=dict(size=22),
            ),
        height=540,
        width=720,
        font=dict(
            family="Open Sans",
            size=14,
        ),
    )
)
crt_pref_fig.add_trace(go.Pie(
    labels = ['Path Drawing', 'Point Placing'], 
    values = crt_pref,
    textinfo='label+percent',
    name='Creation',
    legendgroup="creation",
    legendgrouptitle_text="Creation Method",
    hole=0.3
    ))
colors = [color_uni, color_bi]
crt_pref_fig.update_traces(marker=dict(colors=colors))
crt_pref_fig.show()

In [42]:
crt_pref_fig.write_image("Plots/crt_pref.pdf", engine="kaleido" )

In [43]:
nav_pref_fig = go.Figure(
            layout=go.Layout(
        title = dict(
            text="<b>Participant Navigation Method Preference</b>",
            xanchor='center',
            yanchor='top',
            x=0.5,
            y=0.945,
            font=dict(size=22),
            ),
        height=540,
        width=720,
        font=dict(
            family="Open Sans",
            size=14,
        ),
    )
)
nav_pref_fig.add_trace(go.Pie(
    labels = ['4-DoF', '6-DoF'], 
    values = nav_pref,
    textinfo='label+percent',
    name='Navigation',
    legendgroup="navigation",
    legendgrouptitle_text="Navigation Method",
    hole=.3
    ))
colors = [color_4dof, color_6dof]
nav_pref_fig.update_traces(marker=dict(colors=colors))
nav_pref_fig.show()

In [44]:
nav_pref_fig.write_image("Plots/nav_pref.pdf", engine="kaleido" )

### SSQ Analysis

In [45]:
ssq_pre = participant_data['pre_ssq']
ssq_post = participant_data['post_ssq']
ssq_delta = participant_data['delta_ssq']

n_stat, n_p = stats.shapiro(ssq_delta)
w_stat, w_p = stats.wilcoxon(ssq_delta, nan_policy='omit', zero_method='wilcox')
t_stat, t_p = stats.ttest_rel(ssq_pre, ssq_post, nan_policy='omit')

results_colors = get_results_colors(n_p, w_p, t_p)
results_header = ['<b>Test</b>', '<b>Result</b>', '<b>p-value</b>']
results_values=[['Shapiro-Wilk', 'Wilcoxon', 'T-test'], [n_stat, w_stat, t_stat], [n_p, w_p, t_p]]
results_prefix = ['','', 'p = ',]
fill_color = results_colors

if n_p < 0.05:
    iqr_pre, q1_pre, q3_pre = iqr_stats(ssq_pre)
    iqr_post, q1_post, q3_post = iqr_stats(ssq_post)
    summary_header = ['', '<b>Median</b>', '<b>IQR</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>Pre-SSQ</b>', '<b>Post-SSQ</b>'], [ssq_pre.median(), ssq_post.median()], [iqr_pre, iqr_post], [ssq_pre.min(), ssq_post.min()], [ssq_pre.max(), ssq_post.max()], [ssq_pre.skew(), ssq_post.skew()]]

else:
    summary_header = ['', '<b>Mean</b>', '<b>STD</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>Pre-SSQ</b>', '<b>Post-SSQ</b>'], [ssq_pre.mean(), ssq_post.mean()], [ssq_pre.std(), ssq_post.std()], [ssq_pre.min(), ssq_post.min()], [ssq_pre.max(), ssq_post.max()], [ssq_pre.skew(), ssq_post.skew()]]


In [46]:
stats_table = go.Table(columnwidth=[5,4,4,4,4])
stats_table.header = dict(
    values=summary_header, 
    align = ['right', 'center'],)
stats_table.cells = dict(
        values=summary_values,
        align = ['right', 'center'],
        format=["", ".3f"],
        )

results_table = go.Table(columnwidth=[5,4,4])
results_table.header = dict(
    values= results_header, 
    align = ['right', 'center'],)
results_table.cells = dict(
        values=results_values,
        align = ['right', 'center'],
        format=["", ".2f", ".5f"],
        fill_color=fill_color,
        prefix=results_prefix,
        )

ssq_fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=False,
    specs=[
        [{"type": "table"}, {"type": "box", "rowspan": 2}],
        [{"type": "table"}, None]
    ])

ssq_fig.add_trace(stats_table, row=2, col=1)
ssq_fig.add_trace(go.Box(y=ssq_pre, name="Pre-SSQ", notched=True, notchwidth=0.15), row=1, col=2)
ssq_fig.add_trace(go.Box(y=ssq_post, name="Post-SSQ", notched=True, notchwidth=0.15), row=1, col=2)
ssq_fig.add_trace(results_table, row=1, col=1)

ssq_fig.update_layout(
    title_text="SSQ Weighted Score",
    width=1200,
    yaxis_title='SSQ',
)

ssq_fig.update_yaxes(range=[0,max(1.1*ssq_pre.max(), 1.1*ssq_post.max())])
ssq_fig.show()


In [47]:
ssq_pre_post_fig = go.Figure(
            layout=go.Layout(
        title = dict(
            text="<b>Pre and Post SSQ-Total Scores</b>",
            xanchor='center',
            yanchor='top',
            x=0.5,
            y=0.945,
            font=dict(size=22),
            ),
        yaxis=dict(title="<b>SSQ-Total Score</b>"),
        height=540,
        width=720,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
            ),
        font=dict(
            family="Open Sans",
            size=14,
        ),
    )
)
ssq_pre_post_fig.add_trace(go.Box(y=ssq_pre, name='Pre-SSQ', notched=True, notchwidth=0.25, marker_color='tan', ))
ssq_pre_post_fig.add_trace(go.Box(y=ssq_post, name='Post-SSQ', notched=True, notchwidth=0.25, marker_color='slategray', ))
ssq_pre_post_fig.update_yaxes(range=[0,100])

In [48]:
ssq_pre_post_fig.write_image("Plots/ssq.pdf", engine="kaleido" )